In [ ]:
# All configs

# path to dataset
data_path = "../wasp43b_tsrt"


# path to mjolnir from execution directory, used if ran in another folder, e.g. with 'muninn' script
mjolnir_path = "./"

# figure base size in inches
FIGSIZE_x = 15
FIGSIZE_y = 12

# dots per inch resolution to use to compute figure size
dpi = 96

# video size in pixels
VIDEO_x = 1280
VIDEO_DISPLAY_x = 800

# generate plots needing regridding (slow)
regrided_plots = False

# generate movies (slow)
movies = False

# generate quadruple plots (initial, first output, middle of time, last output)
quad_plots = True

# generate big single plot of last output or selected index
single_plot = True

# for single plot generation, plot last output
plot_last = True

# if not plot last output, plot this index
plot_index = 1000

# plot spectrum of outgoing flux at TOA
plot_spectrum = True

# plot w0 and g0 plots, need to be in output file to work
plot_w0_g0 = False

# plot some diagnostics from diagnostics txt file (needs pandas)
plot_diagnostics = True

# plot some global values from global txt file (needs pandas)
plot_globals = True

output_path = ""




In [ ]:
# path to mjolnir code
import sys
sys.path.append(mjolnir_path) 

import pathlib
import re

import h5py
import imageio
import IPython.display as disp

import math

#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageOps
from matplotlib.backends.backend_agg import FigureCanvasAgg
# from pygifsicle import optimize

import hamarr as ham
from mjolnir_caching_plotter import caching_plotter
from file_helpers import get_path_matching_regex, get_path_matching_regex_with_groups

from datetime import datetime

if plot_diagnostics or plot_globals:
    import pandas as pd

In [ ]:
start = datetime.now()

In [ ]:

if output_path != "":
    output_path = pathlib.Path(output_path)
else:
    output_path = pathlib.Path(data_path) / f"report/"
    

In [ ]:
exp_path = pathlib.Path(data_path) 
FIGSIZE = (FIGSIZE_x, FIGSIZE_y)



In [ ]:
# get planet file

planet_files = get_path_matching_regex_with_groups(
    exp_path, f"esp_output_planet_(.+).h5"
)

if len(planet_files) == 1:
    planet_name = planet_files[0]['groups'][0] 
    planet_file = planet_files[0]['path']
    
    print(f"Found planet name '{planet_name}'")
else:
    raise Exception("Error looking for single planet files. Found:", planet_files)

In [ ]:
# get grid file

grid_files = get_path_matching_regex_with_groups(
    exp_path, f"esp_output_grid_(.+).h5"
)

if len(grid_files) == 1:
    planet_g_name = grid_files[0]['groups'][0] 
    grid_file = grid_files[0]['path']
    
    print(f"Found grid file '{grid_file}'")
else:
    raise Exception("Error looking for single grid files. Found:", grid_files)

In [ ]:
# get diagnostics file

diagnostics_files = get_path_matching_regex_with_groups(
    exp_path, f"esp_diagnostics_(.+).txt"
)

if len(diagnostics_files) == 1:
    planet_d_name = diagnostics_files[0]['groups'][0] 
    diagnostics_file = diagnostics_files[0]['path']
    
    print(f"Found diagnostics file '{diagnostics_file}'")
else:
    raise Exception("Error looking for single diagnostics files. Found:", diagnostics_files)

In [ ]:
# get globals file

globals_files = get_path_matching_regex_with_groups(
    exp_path, f"esp_global_(.+).txt"
)

if len(globals_files) == 1:
    planet_glb_name = globals_files[0]['groups'][0] 
    globals_file = globals_files[0]['path']
    
    print(f"Found globals file '{globals_file}'")
else:
    raise Exception("Error looking for single globals files. Found:", globals_files)

In [ ]:
print()
print("Planet Parameters")
print("=================")
print()

with h5py.File(planet_file, 'r') as p:
    for k in p.keys():
        print(f"{k:<30}: {str(p[k][...]):>15}")
    
    has_TSRT = "two_streams_radiative_transfer" in p and p['two_streams_radiative_transfer'][0] == 1.0
    has_RT = "radiative_transfer" in p and p['radiative_transfer'][0] == 1.0
    
    print()
    print("Modules")
    print("-------")
    print()
    print(f"Radiative transfer: {has_RT}")
    print(f"Two stream radiative transfer: {has_TSRT}")

In [ ]:
print()
print("Grid Parameters")
print("===============")
print()

with h5py.File(grid_file, 'r') as g:
    # print(g.keys())
              
    #for k in g.keys():
    #    print(f"{k:<30}: {str(g[k][...]):>15}")
    print(f"{'nv':<30}: {str(g['nv'][...]):>15}" )
    print(f"{'point_num':<30}: {str(g['point_num'][...]):>15}" )

In [ ]:
# get count of output files
outputs = get_path_matching_regex_with_groups(
    exp_path, f"esp_output_{planet_name}_(\d+).h5"
)

# sort'em
d = {}

for o in outputs:
    p = o["path"]
    (idx_chr,) = o["groups"]
    idx = int(idx_chr)
    # print(f"{idx} - {p}")
    d[idx] = p

sorted_files = sorted(d.items())


first_idx = sorted_files[0][0]
last_idx = sorted_files[-2][0]
#last_idx = 20

print()
print("Output files")
print("============")
print()

print(f"First index: {first_idx}")
print(f"Last index: {last_idx}")

print()
print("Plotting")
print("========")
print()
print(f"Plot quad: {quad_plots}")
print(f"Plot single plot: {single_plot}")
print(f"Plot last: {plot_last}")

if plot_last:
    plot_index = last_idx
print(f"plot_index: {plot_index}" )


In [ ]:
class args:
    pass


file_idx = 1

args.pview = ["fuptot"]
args.file = [str(exp_path)]
args.simulation_ID = [planet_name]
args.initial_file = [file_idx]
args.last_file = [file_idx]
args.horizontal_lev = [2.5e2]
args.vertical_top = ["default"]
args.split_layer = ["no_split"]
args.coordinate_sys = ["icoh"]
args.lmax_adjust = [0]
args.slice = [0, 180]
args.maketable = False
args.no_pressure_log = False
args.latlonswap = False
args.vcoord = ["pressure"]
#args.pgrid_ref = [f"pgrid_{file_idx}_{file_idx}_1.txt"]
args.pgrid_ref = ["auto"]
args.clevels = [40]

In [ ]:
figures_dir = exp_path / "figures"
if not figures_dir.exists():
    figures_dir.mkdir(parents=True, exist_ok=True)
    
if not output_path.exists():
    output_path.mkdir(parents=True, exist_ok=True)

In [ ]:
class multi_plotter:
    """Plotting class to manage multiple plots at individual times over dataset."""
    def __init__(self, args):
        self.args = args
        self.plotters = {}
    
    def load_plotter(self, plot_idx):
        if plot_idx not in self.plotters:
            args.initial_file = [plot_idx]
            args.last_file = [plot_idx]

            stride = 1
            overwrite = False
            #args.pgrid_ref = [f"pgrid_{plot_idx}_{plot_idx}_1.txt"]
            
            self.plotters[plot_idx] = caching_plotter(args)
        return self.plotters[plot_idx]
    
    def plot_steps(self, plot_type, first_idx, last_idx, override_args={}):
        fig = plt.Figure(figsize=FIGSIZE, dpi=dpi)
        ((ax_first, ax_last), (ax_second, ax_mid)) = fig.subplots(2,2)

        i = first_idx
        plotter = self.load_plotter(i)
        plotter.make_plot(plot_type, False, axis=(fig, ax_first), override_args=override_args)
        ttl = ax_first.get_title()
        ax_first.set_title(ttl + "\n" + f"Initial (idx={i})")

        i = last_idx
        plotter = self.load_plotter(i)
        plotter.make_plot(plot_type, False, axis=(fig, ax_last), override_args=override_args)
        ttl = ax_last.get_title()
        ax_last.set_title(ttl + "\n" + f"Last (idx={i})")

        i = first_idx + 1
        plotter = self.load_plotter(i)
        plotter.make_plot(plot_type, False, axis=(fig, ax_second), override_args=override_args)
        ttl = ax_second.get_title()
        ax_second.set_title(ttl + "\n" + f"First step (idx={i})")

        i = (last_idx + first_idx) // 2
        plotter = self.load_plotter(i)
        plotter.make_plot(plot_type, False, axis=(fig, ax_mid), override_args=override_args)
        ttl = ax_mid.get_title()
        ax_mid.set_title(ttl + "\n" + f"Middle step (idx={i})")
    
        return fig
    
    def plot_single(self, plot_type, plot_idx, override_args={}):
        fig = plt.Figure(figsize=(FIGSIZE_x, FIGSIZE_y), dpi=dpi)
        ax = fig.subplots(1,1)

        plotter = self.load_plotter(plot_idx)
        plotter.make_plot(plot_type, False, axis=(fig, ax), override_args=override_args)
  
        ttl = ax.get_title()
        ax.set_title(ttl + "\n" + f"idx={plot_idx}")
        
        return fig
    
    def plot_single_band(plot_type, plot_idx, override_args={}):
        fig = plt.Figure(figsize=(4*FIGSIZE_x, 4*FIGSIZE_y), dpi=dpi)
    
        i = first_idx
        plotter = self.load_plotter(i)
        plotter.make_plot(plot_type, False, axis=(fig,), override_args=override_args)
        
        return fig
    
    
    def plot_anim_steps(self, plot_type, first_idx, last_idx, plot_filename, overwrite_anim=False, override_args={}):
        """Make a movie using args for mjolnir plotting functions, run over all the indexed files"""
        stride = 1
        overwrite = False
        output_anim_file = output_path / plot_filename

        if output_anim_file.exists() and not overwrite_anim:
            print(
                f"{output_anim_file} already exists, skipping. Set overwrite_anim = True to force"
            )
            return output_anim_file

        fps = 10
        # writer = imageio.get_writer(str(output_image), fps=fps, quality=10)
        writer = imageio.get_writer(str(output_anim_file), fps=fps)

        # for dev, force to use a small number of files
        # last_idx  = 20
        # compute fig size so that it gets rounded to closest multiple of video macro_block_size
        block_size = 16
        fs_x = (math.ceil((FIGSIZE_x*dpi)/float(block_size))*block_size)/dpi
        fs_y = (math.ceil((FIGSIZE_y*dpi)/float(block_size))*block_size)/dpi
    
        fig = plt.Figure(figsize=(fs_x, fs_y), dpi=dpi)
        ax = fig.subplots(1, 1)
    
        #size=(VIDEO_x, int(VIDEO_x/FIGSIZE_x*FIGSIZE_y))
        
        

        for i in range(first_idx, last_idx + 1):

            print(f"plotting: {i: 5}/{last_idx}\r", end="")
            
            fig.clear()
            ax = fig.add_subplot(111)
    
            plotter = self.load_plotter(i)
            plotter.make_plot(plot_type, False, axis=(fig, ax), override_args=override_args)
  
            
        
            # canvas = fig.canvas
            canvas = FigureCanvasAgg(fig)

            # Option 2: Retrieve a view on the renderer buffer...
            canvas.draw()
            buf = canvas.buffer_rgba()
            # ... convert to a NumPy array ...
            X = np.asarray(buf)
            # ... and pass it to PIL.
            # im = Image.fromarray(X)

            #import pdb; pdb.set_trace()
            writer.append_data(X)

        writer.close()

        # optimize gif output
        #optimize(str(output_image))

        return output_anim_file



    
    def plot_anim(self, plotname, plotfile, first, last, override_args={}):
        print(f"plotting {plotname} to {plotfile}")
       
        # don't show interactive plots when not asked to

        output_image = self.plot_anim_steps(plotname, first_idx, last_idx, plotfile, override_args=override_args)

        print("video "+str(output_image))
        #im = Image.open(output_image)
        #return disp.Video(data=str(output_image),width=VIDEO_DISPLAY_x)
        return disp.HTML(f"""<video alt="{plotname} plot" width={VIDEO_DISPLAY_x} controls>
                             <source src="{output_image}" type="video/mp4">
                             </video>
                             """)


In [ ]:
# make list of possible plots, so that hamarr knows if it has to regrid
args.pview = ["TP", "spectrum"]

if regrided_plots:
    args.pview += ["Tver", "Tlonver", "uver"]
        
if has_TSRT:
    args.pview += ["qheatprof",
                   "qheat",
                   "TSqheatprof",
                   "DGqheatprof",
                   "TSfutprof",
                   "TSfdtprof",
                   "TSfluxprof",
                   "w0prof", 
                   "g0prof",
                   "TSfdirprof"
                  ]
    
    if regrided_plots:
        args.pview += ["TSfuptot", "TSfdowntot", "TSfnet"]
    
if plot_spectrum:
    args.pview += ['spectrum']


mp = multi_plotter(args)

# Mu Star

In [ ]:
f = None
if has_TSRT:
    f = mp.plot_steps("mustar", first_idx, last_idx)
f

# TP Profile

In [ ]:
pa = None
if movies:
    pa = mp.plot_anim("TP", "TP_anim.mp4", first_idx, last_idx)
pa

In [ ]:
f = None
if quad_plots:
    f = mp.plot_steps("TP", first_idx, last_idx)
f

In [ ]:
f = None
if single_plot:
    f = mp.plot_single("TP", plot_index)
f

# Tver

In [ ]:
pa = None
if movies and regrided_plots:
    pa = mp.plot_anim("Tver", "Tver_anim.mp4", first_idx, last_idx)
pa

In [ ]:
f = None
if quad_plots and regrided_plots:
    f = mp.plot_steps("Tver", first_idx, last_idx)
f

In [ ]:
f = None
if single_plot and regrided_plots:
    f = mp.plot_single("Tver", plot_index)
f

# Tlonver

In [ ]:
pa = None
if movies and regrided_plots:
    pa = mp.plot_anim("Tlonver", "Tlonver_anim.mp4", first_idx, last_idx)
pa


In [ ]:
f = None
if regrided_plots and quad_plots:
    f = mp.plot_steps("Tlonver", first_idx, last_idx)
f

In [ ]:
f = None
if single_plot and regrided_plots:
    f = mp.plot_single("Tlonver", plot_index)
f

# Zonal wind profile ulev

In [ ]:
pa = None
if movies and regrided_plots:
    pa = mp.plot_anim("uver", "uver_anim.mp4", first_idx, last_idx)
pa

In [ ]:
f = None
if quad_plots and regrided_plots:
    f = mp.plot_steps("uver", first_idx, last_idx)
f

In [ ]:
f = None
if single_plot and regrided_plots:
    f = mp.plot_single("uver", plot_index)
f

# Two Stream Radiative transfer

In [ ]:
if has_TSRT:
    print("Two Stream Radiative Transfer Enabled")
else:
    print("Two Stream Radiative Transfer Disabled, no plots will appear in this group")

## Qheat

### Horizontal qheat

In [ ]:
pa = None
if has_TSRT and movies and regrided_plots:
    pa = mp.plot_anim("qheat", "qheat_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots and regrided_plots:
    f = mp.plot_steps("qheat", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("qheat", plot_index)
f

### Combined qheat
This is the combined Qheat computed if running with Double Gray spin up transitioning to Two Streams. 

In [ ]:
pa = None
if has_TSRT and movies:
    pa = mp.plot_anim("qheatprof", "qheatprof_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots:
    f = mp.plot_steps("qheatprof", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("qheatprof", plot_index)
f

### Two Streams qheat

In [ ]:
pa = None
if has_TSRT and movies:
    pa = mp.plot_anim("TSqheatprof", "TSqheatprof_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots:
    f = mp.plot_steps("TSqheatprof", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("DGqheatprof", plot_index)
f

In [ ]:
pa = None
if has_TSRT and has_RT and movies:
    pa = mp.plot_anim("DGqheatprof", "DGqheatprof_anim.mp4", first_idx, last_idx)
    
pa

## Total Upward Flux 

### Profile

In [ ]:
pa = None
if has_TSRT and movies:
    pa = mp.plot_anim("TSfutprof", "futprof_anim.mp4", first_idx, last_idx)

pa

In [ ]:
f = None
if has_TSRT and quad_plots:
    f = mp.plot_steps("TSfutprof", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("TSfutprof", plot_index)
f

### Horizontal

In [ ]:
pa = None

if has_TSRT and movies and regrided_plots:
    pa = mp.plot_anim("TSfuptot", "fuptot_anim.mp4", first_idx, last_idx, {"horizontal_lev": [1e-2]})
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots and regrided_plots:
    f = mp.plot_steps("TSfuptot", first_idx, last_idx)
    
f

In [ ]:
f = None
if has_TSRT and single_plot and regrided_plots:
    f = mp.plot_single("TSfuptot", plot_index)
f

## Total Downward Flux 

### Profile

In [ ]:
pa = None
if has_TSRT and movies:
    pa = mp.plot_anim("TSfdtprof", "fdtprof_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots:
    f = mp.plot_steps("TSfdtprof", first_idx, last_idx)
f


In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("TSfdtprof", plot_index)
f

### Horizontal

In [ ]:
pa = None
if has_TSRT and movies and regrided_plots:
    pa = mp.plot_anim("TSfdowntot", "fdowntot_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots and regrided_plots:
    f = mp.plot_steps("TSfdowntot", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot and regrided_plots:
    
    f = mp.plot_single("TSfdowntot", plot_index, override_args={"horizontal_lev": [2.5e4]})
f

## Total Net Flux 

### Profile

In [ ]:
pa = None 

if has_TSRT and movies:
    pa = mp.plot_anim("TSfluxprof", "fluxprof_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots:
    f = mp.plot_steps("TSfluxprof", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("TSfluxprof", plot_index)
f

### Horizontal

In [ ]:
pa = None
if has_TSRT and movies and regrided_plots:
    pa = mp.plot_anim("TSfnet", "TSfnet_anim.mp4", first_idx, last_idx)
    
pa

In [ ]:
f = None
if has_TSRT and quad_plots and regrided_plots:
    f = mp.plot_steps("TSfnet", first_idx, last_idx)
f

In [ ]:
f = None
if has_TSRT and single_plot and regrided_plots:
    f = mp.plot_single("TSfnet", plot_index)
f

## Direct Beam Flux

### Profile

In [ ]:
f = None
if has_TSRT and single_plot:
    f = mp.plot_single("TSfdirprof", plot_index)
f

## omega0 and g0

In [ ]:



f = None
if has_TSRT and single_plot and plot_w0_g0:
    f = mp.plot_single_band("w0prof", plot_index)
f


In [ ]:

f = None
if has_TSRT and single_plot and plot_w0_g0:
    f = mp.plot_single_band("g0prof", plot_index)
f

## Spectrum

Upward flux at TOA, incoming stellar flux envelope

In [ ]:
f = None
if has_TSRT and plot_spectrum:
    f = mp.plot_single("spectrum", plot_index)
f

# Globals
Simple dump of content of globals file

In [ ]:
fig = None
if plot_globals:
    global_data = pd.read_csv(globals_file, sep='\s+')
    
    fig = plt.Figure(figsize=FIGSIZE, dpi=dpi)
    ((ax_E, ax_M, ax_ent), (ax_AMx, ax_AMy, ax_AMz), (ax_AM, ax_dummy1, ax_dummy2)) = fig.subplots(3,3)
    ax_E.plot(global_data["simulation_time"]/(3600*24), global_data["GlobalE_h"])
    ax_E.set_title("GlobalE")
    ax_E.grid(True)
    ax_E.set_xlabel("Time [days]")
    
    ax_M.plot(global_data["simulation_time"]/(3600*24), global_data["GlobalMass_h"])
    ax_M.set_title("GlobalMass")
    ax_M.grid(True)
    ax_M.set_xlabel("Time [days]")
    
    ax_ent.plot(global_data["simulation_time"]/(3600*24), global_data["GlobalEnt_h"])
    ax_ent.set_title("GlobalEnt")
    ax_ent.grid(True)
    ax_ent.set_xlabel("Time [days]")
    
    ax_AMx.plot(global_data["simulation_time"]/(3600*24), global_data["GlobalAMx_h"])
    ax_AMx.set_title("GlobalAMx")
    ax_AMx.grid(True)
    ax_AMx.set_xlabel("Time [days]")
    
    ax_AMy.plot(global_data["simulation_time"]/(3600*24), global_data["GlobalAMy_h"])
    ax_AMy.set_title("GlobalAMy")
    ax_AMy.grid(True)
    ax_AMy.set_xlabel("Time [days]")
    
    ax_AMz.plot(global_data["simulation_time"]/(3600*24), global_data["GlobalAMz_h"])
    ax_AMz.set_title("GlobalAMz")
    ax_AMz.grid(True)
    ax_AMz.set_xlabel("Time [days]")
    
    ax_AM.plot(global_data["simulation_time"]/(3600*24), np.sqrt(np.power(global_data["GlobalAMx_h"],2.0) + np.power(global_data["GlobalAMy_h"],2.0) + np.power(global_data["GlobalAMz_h"],2.0)) )
    ax_AM.set_title("GlobalAM")
    ax_AM.grid(True)
    ax_AM.set_xlabel("Time [days]")
    
    ax_dummy1.set_visible(False)
    ax_dummy2.set_visible(False)
fig

# Diagnostics

In [ ]:
fig = None
if plot_diagnostics:
    diagnostics_data = pd.read_csv(diagnostics_file, sep='\s+')
    
    fig = plt.Figure(figsize=FIGSIZE, dpi=dpi)
    ((ax_time), (ax_mean_per_step)) = fig.subplots(2,1)
    ax_time.plot(diagnostics_data['#current_step'], diagnostics_data['elapsed_time'])
    ax_time.grid(True)
    ax_time.set_xlabel("step #")
    ax_time.set_ylabel("time [s]")
    ax_time.set_title("elapsed time since start [s]")
    
    ax_mean_per_step.plot(diagnostics_data['#current_step'], diagnostics_data['mean_delta_per_step'])
    ax_mean_per_step.grid(True)
    ax_mean_per_step.set_xlabel("step #")
    ax_mean_per_step.set_ylabel("delta time [s]")
    ax_mean_per_step.set_title("mean estimated time per step [s]")
    

    
    
fig 

## Timing info

In [ ]:
stop = datetime.now()

delta = stop - start

print(f"Ran plotting script in {delta}")